# **AVID** (**A**utomated processing of **V**ersatile **I**mage **Da**ta) in a nutshell:

### Purpose

AVID is a **data processing tool for automated image analysis** which for example allows for 
- running a processing pipeline with as little manual user interaction as possible in order to avoid tedious, error prone manual analysis workflows.
- batch-processing of a large bunch of data, e.g. registration of CT and MR data for a large cohort of patients at multiple time points.
- repeatedly running a workflow to update results (e.g. when new datasets arrive on a PACS system).
- automated application and orchestration of command line tools like MITK command apps.

### Advantages
- **simple**: AVID is a light-weight application in the sense of compact and flexible workflows.
- **portable**: The AVID workflows are operating system independent and can be easily deployed as Python script or as executable.
- **data-driven & scalable**: The data-driven approach enables AVID to automatically scale to the data from single cases to cohorts.
- **extendable**: New tools can be easily added.

### Technical note

AVID is written in Python. Python 3 or higher is required.  



# Basic concepts

AVID is a tool that facilitates the composition of workflows for data analysis. A variety of processing steps can be combined to obtain the overall workflow.

The concrete flow of the data through the workflow and how the data is combined as input for the execution of individual processing steps is primarily determined by the properties of the data itself, rather than being defined by a fixed set of algorithms. 
This makes AVID automatically scalable to a variety of processing scenarios. It doesn't matter if a single case or a cohort is being processed, or if the image modalities, or the number of sequences or timepoints vary. This makes AVID very flexible in regard to the varying demands of medical image processing.      
<br>
This is realized by the following design:


<img src="AVIDDesign.PNG" width="681" height="342" />



### Data 
This is the dataset which is being processed in the workflow. It contains the "raw" input data to the workflow and the output data of each processing step. The data can be located for example in a data folder or a database such as a PACS system. Each data item within the dataset has a URL pointing to where it is located.

### Session 
The session is the central data repository of AVID. It can be seen as a "blackboard" which contains all the relevant metadata about the *data* and the processing pipeline in the form of *artefacts*. It can be read from and written to by the *actions*. The user can directly feed information about the initial input data to the session in the form of an xml file. It is also possible to gain insights into the current session by writing out the artefacts as items of an xml-file.

### Artefacts 
In AVID data is handled in the form of artefacts. Each artefact is stored as an item of the session and refers to a specific data entry, e.g. a data folder. It contains relavant metadata referred to as *properties* such as the patient case, time point, data format as well as the URL to where the data entry is located.<br>
The artefacts of a session can be written out in an xml-file. An exemplary artefact looks like this:<br>
```xml
<avid:artefact>
    <avid:property key="case">pat1</avid:property>
    <avid:property key="timePoint">TP1</avid:property>
    <avid:property key="actionTag">CT</avid:property>
    <avid:property key="type">result</avid:property>
    <avid:property key="format">itk</avid:property>
    <avid:property key="url">../data/img/pat1_TP1_CT.txt</avid:property>
    <avid:property key="objective">CT</avid:property>
    <avid:property key="invalid">False</avid:property>
    <avid:property key="id">bbe232b8-5740-11ec-85a6-e9d058c65a83</avid:property>
    <avid:property key="timestamp">1638869608.3330662</avid:property>
</avid:artefact>
```

### Actions
Actions make things happen in AVID. Each action corresponds to a specific independent step of the data processing pipeline. <br>
Various actions are available in AVID in the form of Python sripts and command line interface (CLI) applications. A variety of CLI applications are provided in the form of the so-called *Cmd-Apps*. Cmd-Apps are CLI applications imported from specific tool-packages, such as the Medical Imaging Interaction Toolkit (MITK). 
Actions can do all sorts of things, including fitting, registration, resampling. The available actions implemented as Python scripts are located in the folder `AVID\avid\actions`. AVID can be straightforwardly extended to new actions if a required action is not available.<br>
Once an action has been called, the blackboard will be updated and in case new data have been generated by the action, new artefacts will be added. In this context, the artefact property *actionTag* exists which describes the action that has created a data entry. If *action_1* has created new processed data, the default actionTag of the corresponding artefact will look like this:
```xml
<avid:property key="actionTag">action_1</avid:property>
```


### Workflow script
The workflow script is a Python script which orchestrates the interaction between the session and the actions. It determines which actions to activate and in which order to activate them. The workflow script can be triggered by the data, e.g. it can be called when new data arrives in a database. All currently available workflow scripts can be found in the repository https://git.dkfz.de/mic/internal/avid-workflows. AVID can be easily extended to new workflow scripts.

### Selector 
Data are not explicitly handed to actions. Instead, *selectors* are *linkers* used. They allow us to specify which artefacts should be used as input to an action. This way, instead of using all currently available artefacts from the session we can choose only a selection. A selector selects artefacts based on properties. For example, we can tell the selector of `action_2` to perform the action only on artefacts with the property `ActionTag`=`action_1`. Then action_2 is performed only on artefacts which have been generated by action_1. All available selectors are located in the folder `AVID\avid\selectors`. 

### Linker
Actions can also be given more than a single input, meaning they don't have to work on individual artefacts, but can also work on pairs of artefacts (or even more). For example we could wish to perform a registration of MR and CT images, which takes both images as input. We can use a selector to select the desired images, but there is a problem. How do we clarify which artefacts belong together in a pair? Theoretically, each MR image could be paired with each CT image, across patients and time points. To get exactly what we want, we use *linkers*. All available linkers are located in the folder `AVID\avid\linkers`. 

### Splitter
*Splitters* can be used to split an artefact list by certain criteria and return the splitted artefact lists. For example an artefact list can be split by a *splitter* in such a way that all artefacts from the same case are grouped together in one split. 

### Sorter
*Sorters* can be used to sort an artefact list by certain criteria and pass back the sorted list. For example an artefact list can be sorted using a *sorter* such that the artefacts in the list are sorted by the time point property of the artefacts.


### Datacrawler
To start the data-driven workflow, there needs to be an initial *bootstrap session* which contains artefacts of the input data to the processing pipeline. This is generated by a Python script typically called ```datacrawler.py```. The datacrawler generates an xml-file which can be read in by the session during the initialization of a workflow. Typically a datacrawler script can be found along with a workflow script. Usually, the datacrawler assumes a certain pre-defined naming convention of the data folders in order to set the properties.
Remark: Using the datacrawler is a design choice here. The bootstrap artefacts could also be provided to the session in a different way.


# Basic Example:

In this easy example we will go over the above described basic concepts with a concrete example. We will learn how to 
- run a datacrawler to generate bootstrap artefacts
- run actions
- chain actions together
- use a selector to apply actions onto to a selection of data
- use a linker to apply actions to a combination of data

*Remark: Before running this example please make sure the ```examples\output``` folder is empty. This is especially relevant in case you run the whole example multiple times. Additionally, be aware to only execute the cells in the given order and not to re-run cells multiple times since this might lead to errors or change the example output.*

## Example Data:

Assume the following example data set found at ```examples\data\img```:

We have data from 2 patients "pat1" and "pat2", acquired at different time points "TP1" and "TP2" with the different modalities "MR" and "CT": 
- Patient1
  - pat1_TP1_CT.txt
  - pat1_TP1_MR.txt
  - pat1_TP2_MR.txt
- Patient2
    - pat2_TP1_CT.txt
    - pat2_TP1_MR.txt

For the purpose of this exercise each of these files initially consists of empty text files and the actions will write text into them. However, the same concepts would apply when applying more complex image processing actions to actual image data.

We use the naming convention `[case]_[timepoint]_[actiontag]` which encodes the properties that are used in this example. For the initial raw data we set the actionTag to *CT* or *MR* since the modality can be interpreted as an "action" that generates the images.

## Bootstrap Artefacts and Datacrawler

As a first step we need to create the bootstrap artefacts. More specifically, in our case it is an XML file containing a collection of artefact items for each input dataset, serving as a starting point for our pipeline.

The bootstrap artefacts XML file is generated by running the script ```examples\datacrawler.py```. It takes as input
- the data root folder, in this example: ```data/```
- the output file path, in this example: ```output/bootstrap.avid```

As the name implies the datacrawler "crawls" through the specified root folder and its subfolders and creates an artefact item for each file it encounters. Some of the metadata are set based on the naming scheme of the files, e.g. in our case the naming convention `[case]_[timepoint]_[actiontag]` is used to set the *case*, the *timePoint* and the *actionTag* property. 

Typically, the datacrawling process needs to be done only one time even if the pipeline is executed multiple times. This is useful since for large datasets generating the bootstrap artefacts might be time-consuming.
In practice, when using AVID, the bootstrap artefacts might be already provided by us. Then you can skip this step.

In [22]:
###############################################################################
# Run Datacrawler
###############################################################################
%run datacrawler.py data/ output/bootstrap.avid


Check "data/img\pat1_TP1_CT.txt": Added
Check "data/img\pat1_TP1_MR.txt": Added
Check "data/img\pat1_TP2_MR.txt": Added
Check "data/img\pat2_TP1_CT.txt": Added
Check "data/img\pat2_TP1_MR.txt": Added
Finished crawling. Number of generated artefacts: 5


*Result*: The output file ```examples\output\bootstrap.avid``` contains artefact items for each text file found in the ```examples\data``` folder, including the subfolder ```examples\data\img```.

For example, the corresponding artefact item of pat1_TP1_CT.txt looks something like this (with a varying timestamp):<br>
```xml
  <avid:artefact>
    <avid:property key="case">pat1</avid:property>
    <avid:property key="timePoint">TP1</avid:property>
    <avid:property key="actionTag">CT</avid:property>
    <avid:property key="type">result</avid:property>
    <avid:property key="format">itk</avid:property>
    <avid:property key="url">../data/img/pat1_TP1_CT.txt</avid:property>
    <avid:property key="objective">CT</avid:property>
    <avid:property key="invalid">False</avid:property>
    <avid:property key="id">bbe232b8-5740-11ec-85a6-e9d058c65a83</avid:property>
    <avid:property key="timestamp">1638869608.3330662</avid:property>
  </avid:artefact>
```

## Required Libraries

Ok, now that we have the bootstrap artefacts, we are ready to run our workflow step-by-step.<br>
First, we import all the libraries we need. Specifically, we import the workflow script, actions, selectors and linkers we need for this example. We will dive deeper into the details of the specific actions, selectors and linkers once we encounter them again in the example workflow. <br>  

In [24]:
###############################################################################
# Imports
###############################################################################
import os
import avid.common.workflow as workflow

from avid.actions.pythonAction import PythonUnaryBatchAction, PythonBinaryBatchAction
from avid.selectors import ActionTagSelector, CaseSelector, ValiditySelector
from avid.linkers import CaseLinker, TimePointLinker

## Workflow Initialization

Next, we need to initialize the workflow session by
- loading the previously generated bootstrap artefact XML file into the workflow
- setting the session path, in this example ```\output\output.avid```. This is the place where all artefacts of the whole session, including the bootstrap artefacts and all the artefacts generated via actions, will be stored. It is set to a different file than the bootstrap artefact since it can be a good idea to separate the data crawling from the actual session workflow since then the potentially time-consuming datacrawling process has to be run only one time even if the workflow is restarted.
- setting the session name, here to "example_session"
- setting some optional flags, e.g. for debugging purposes 

In [26]:
###############################################################################
# Initialize session with existing Artefacts
###############################################################################
session =  workflow.initSession(bootstrapArtefacts=os.path.join(os.getcwd(),'output', 'bootstrap.avid'),
                                sessionPath=os.path.join(os.getcwd(),'output', 'output.avid'),
                                name = "example_session",
                                expandPaths=True,
                                debug=True,
                                autoSave = True)

## Example 1
Now we want to do something with the initial artefacts. Remeber from the basic concepts: What makes things happen in AVID are *actions*.
As a start, we will define a simple self-written action that calls a specified Python-function `write_filename` for each input artefact which writes into an empty input text file "Result for file is (name of file)". <br> 

In [28]:
def write_filename(outputs, inputs, **kwargs):
    """
        Simple callable that outputs a sentence including the filename of the input
    """
    inputName = os.path.basename(inputs[0])
    
    with open(outputs[0], "w") as ofile:
        ofile.write(f"Result for file is '{inputName}'")

Now we want to apply this action to our artefacts. We do this by using the ```PythonUnaryBatchAction```. We use the *Unary* batch action because we assume only one input artefact at a time will be passed to the script.<br> 
*Spoiler*: Later on we will find that also more than one input artefact can be passed to an action e.g. by using the ```PythonBinaryBatchAction```.<br>

The ```PythonUnaryBatchAction``` requires as input
- *inputSelector*: We need to specify which artefacts we want to apply the action onto. For that the *selectors* comes into play. Let's start with the simple case that we want to apply the action to all files that are valid. Let's recall what an artefact looks like and remember that one of the properties is the *invalid* property e.g.
    ```xml
      <avid:property key="invalid">False</avid:property>
    ``` 
    We use the already pre-defined ```ValiditySelector``` which only selects the artefacts for which the invalid property is false.
- *generateCallable*: The action we want to use, in our case ```write_filename```.
- *actionTag*: The name of the *actionTag* of all artefacts produced by this action, in this example *example1*. If no name is specified, by default it will just be the name of the action (here: *PythonUnaryBatchAction*)
- *defaultoutputextension*: The file extension of the files produced by this action. In this example, the resulting data will be stored as "txt" files.  

In [30]:
allValid_selector = ValiditySelector()


with session:
    PythonUnaryBatchAction(
        inputSelector=allValid_selector,
        generateCallable=write_filename,
        actionTag="example1",
        defaultoutputextension="txt"
    ).do()


2024-11-06 12:45:38,313 [INFO] Starting action: PythonUnaryBatchAction_example1 (UID: eca8e36e-5a4b-46b6-a2a4-1fc7a6b2f22b) ...
2024-11-06 12:45:38,314 [INFO] Starting action: write_filename (UID: 8d463ce3-5e70-4ccd-b91a-84aece99ed5e) ...
2024-11-06 12:45:38,318 [INFO] Finished action: write_filename (UID: 8d463ce3-5e70-4ccd-b91a-84aece99ed5e) -> SUCCESS
2024-11-06 12:45:38,319 [INFO] Starting action: write_filename (UID: 1bbfb468-7ec0-4203-8cd3-3b5c81b99ecf) ...
2024-11-06 12:45:38,321 [INFO] Finished action: write_filename (UID: 1bbfb468-7ec0-4203-8cd3-3b5c81b99ecf) -> SUCCESS
2024-11-06 12:45:38,325 [INFO] Starting action: write_filename (UID: 28bcfa04-5225-425e-91cb-cd6644ae53cb) ...
2024-11-06 12:45:38,327 [INFO] Finished action: write_filename (UID: 28bcfa04-5225-425e-91cb-cd6644ae53cb) -> SUCCESS
2024-11-06 12:45:38,328 [INFO] Starting action: write_filename (UID: 3c015b92-65a3-4904-9d89-25114913cbe2) ...
2024-11-06 12:45:38,330 [INFO] Finished action: write_filename (UID: 3c015

*Result*: Let's have a look at the resulting output in the ```examples\output``` folder:
- The file ```examples\output\output.avid```has been created. As we have specified in the initialization this is the session path. It is another XML file containing artefact items, just as ```bootstrap.avid```. Along with the bootstrap artefact items it contains additionally all the artefacts generated from the action run in example 1, which all have the actionTag *example1*. Here is an example of a newly created artefact item:
```xml
  <avid:artefact>
    <avid:property key="case">pat1</avid:property>
    <avid:property key="timePoint">TP1</avid:property>
    <avid:property key="actionTag">example1</avid:property>
    <avid:property key="type">result</avid:property>
    <avid:property key="format">itk</avid:property>
    <avid:property key="url">example_session/example1/result/pat1/write_filename.cdcadf1a-9b68-11ef-a1a2-f894c218a9f1.txt</avid:property>
    <avid:property key="objective">CT</avid:property>
    <avid:property key="invalid">False</avid:property>
    <avid:property key="input_ids">
      <avid:input_id key="inputs">b3694134-9b68-11ef-b3e7-f894c218a9f1</avid:input_id>
    </avid:property>
    <avid:property key="action_class">PythonAction</avid:property>
    <avid:property key="action_instance_uid">783d5956-d137-4399-9384-ec13b4fa25dc</avid:property>
    <avid:property key="id">cdcadf1a-9b68-11ef-a1a2-f894c218a9f1</avid:property>
    <avid:property key="timestamp">1730805990.330345</avid:property>
    <avid:property key="execution_duration">0.001995563507080078</avid:property>
  </avid:artefact>
  ```
Compared to the bootstrap artefacts the properties list is extended to additional properties describing the action execution such as the *input ids* or the *execution_duration*. 

- A new folder "example_session" has been created (recall that we have set "example_session" as "name" is the initialization of the session). Here, the resulting files created by the action are stored. For each applied action a new subfolder will be created with the name of the actionTag. Therefore, the resulting files of "example1" can be found in the folder ```output\example_session\example1```. Specifically, they are text files for which the original name of the input files is written into the file as text.

## Example 2:
Let's use the output we produced in the previous example and apply a different action. For that, we define another self-written action that calls the Python-function `extend_content` for each input artefact which reads the content of an input file and writes a new text file also including the previous content.

In order to chain actions together, we use the *ActionTagSelector*. By setting the ActiontagSelector to "example1", only artefacts generated in the previous example will be considered for the next action.

This time we also wish to select only a smaller portion of these artefacts using an additional *selector*: let's call the function `extend_content` only for the entries of patient 1. For this we use the CaseSelector which is based on the `case` property and set it to "pat1".<br>
We can logically combine these two selectors by using the "+"-operator. Now only data with the case property *'pat1'* AND the ActionTag *'example1'* will be processed.

In [33]:
def extend_content(outputs, inputs):
    """
        Simple callable that reads the content of an input file and writes a new file, including the previous content
    """
    inputName = os.path.basename(inputs[0])
    
    with open(inputs[0], "r") as ifile:
        content = ifile.read()
    with open(outputs[0], "w") as ofile:
        ofile.write(f"New content based on '{inputName}'\nOriginal content: '{content}'")

In [35]:
pat1_selector = CaseSelector('pat1') + ActionTagSelector('example1')

with session:
    PythonUnaryBatchAction(
        inputSelector=pat1_selector,
        actionTag="example2",
        generateCallable=extend_content,
        defaultoutputextension="txt"
    ).do()

2024-11-06 12:45:44,839 [INFO] Starting action: PythonUnaryBatchAction_example2 (UID: 42931fe4-143a-4925-961c-5cada43d5aa7) ...
2024-11-06 12:45:44,840 [INFO] Starting action: extend_content (UID: 65e8665e-497f-406f-b161-e3bdfae37437) ...
2024-11-06 12:45:44,843 [INFO] Finished action: extend_content (UID: 65e8665e-497f-406f-b161-e3bdfae37437) -> SUCCESS
2024-11-06 12:45:44,844 [INFO] Starting action: extend_content (UID: d0126fe6-c25b-454d-a381-625d5e053c67) ...
2024-11-06 12:45:44,847 [INFO] Finished action: extend_content (UID: d0126fe6-c25b-454d-a381-625d5e053c67) -> SUCCESS
2024-11-06 12:45:44,847 [INFO] Starting action: extend_content (UID: cbdfec36-19ab-4eac-9821-6e4b75a95d0f) ...
2024-11-06 12:45:44,850 [INFO] Finished action: extend_content (UID: cbdfec36-19ab-4eac-9821-6e4b75a95d0f) -> SUCCESS
2024-11-06 12:45:44,850 [INFO] Finished action: PythonUnaryBatchAction_example2 (UID: 42931fe4-143a-4925-961c-5cada43d5aa7) -> SUCCESS
2024-11-06 12:45:44,857 [INFO] Successful actions:

*Result*: The resulting text files can be found in ```examples/output/example_session/example2```. We find that text files have only been generated for patient 1, not for patient 2. 

When looking at the ```examples/output/output.avid```we find that the content has been extended by the artefact items with the actionTag "example2".

## Example 3
Actions can also be given more than a single input, meaning they don't have to work on individual artefacts, but can also work on pairs of artefacts (or even more). In case of two inputs we can use the ```PythonBinaryBatchAction```.

In this example, we want to pair up MR images and CT images of the same patient for both, "pat1" and "pat2", using the function ```pair_two_images```, which writes both filenames as text into a new text file. We use the ActionTagSelectors to select the ActionTags 'MR' and 'CT' from the bootstrap artefacts. But there is a problem. How do we clarify which artefacts belong together in a pair? Theoretically, each MR image could be paired with each CT image, across patients and time points. To get exactly what we want, there are *Linkers*. In our case, the *CaseLinker* will ensure pairs will only be created between artefacts that share the same case.

In [38]:
def pair_two_images(inputs1, inputs2, outputs):
    """
        Simple callable that outputs the names of the two inputs
    """
    text = f"Matched up two images.  Input 1: {os.path.basename(inputs1[0])}  Input 2: {os.path.basename(inputs2[0])}"
    
    with open(outputs[0], "w") as ofile:
        ofile.write(text)

In [40]:
mr_selector = ActionTagSelector('MR')
ct_selector = ActionTagSelector('CT')


with session:
    PythonBinaryBatchAction(
        inputs1Selector=mr_selector,
        inputs2Selector=ct_selector,
        inputLinker=CaseLinker(),
        actionTag="example3",
        generateCallable=pair_two_images,
        defaultoutputextension="txt"
    ).do()

2024-11-06 12:47:34,655 [INFO] Starting action: PythonBinaryBatchAction_example3 (UID: 116592eb-8e83-4fcf-b966-ac9c32d3a2b7) ...
2024-11-06 12:47:34,657 [INFO] Starting action: pair_two_images (UID: 1a2e6e96-7a23-4a90-861d-77ab4c83d08a) ...
2024-11-06 12:47:34,660 [INFO] Finished action: pair_two_images (UID: 1a2e6e96-7a23-4a90-861d-77ab4c83d08a) -> SUCCESS
2024-11-06 12:47:34,661 [INFO] Starting action: pair_two_images (UID: 1b6b3641-4a57-44ab-a3b6-331c294562da) ...
2024-11-06 12:47:34,663 [INFO] Finished action: pair_two_images (UID: 1b6b3641-4a57-44ab-a3b6-331c294562da) -> SUCCESS
2024-11-06 12:47:34,664 [INFO] Starting action: pair_two_images (UID: b729c210-07b7-4871-8ebc-f8fda65cec11) ...
2024-11-06 12:47:34,667 [INFO] Finished action: pair_two_images (UID: b729c210-07b7-4871-8ebc-f8fda65cec11) -> SUCCESS
2024-11-06 12:47:34,668 [INFO] Finished action: PythonBinaryBatchAction_example3 (UID: 116592eb-8e83-4fcf-b966-ac9c32d3a2b7) -> SUCCESS
2024-11-06 12:47:34,676 [INFO] Successful 

*Result*: Looking into the output folder ```examples/output/example_session/example3```, we can see the pairs that were matched up. 

For pat2, there is one MR and one CT image both at TP1, which were matched as: 
- ```Input 1: pat2_TP1_MR.txt  Input 2: pat2_TP1_CT.txt.```
All good here.

For pat1, there are two results: 
- ```Input 1: pat1_TP1_MR.txt  Input 2: pat1_TP1_CT.txt```
- ```Input 1: pat1_TP2_MR.txt  Input 2: pat1_TP1_CT.txt```
  
We can see, that the CT image for timepoint 1 is matched up twice, with the MR images of timepoint 1 and 2. This might not be what we want and instead it could be more meaningful to match only data from the same patient of the same timepoint.

## Example 4
Let's see in this example how we can achieve to only match images of the same patient acquired at the same timepoint.
To achieve that we can combine different linkers like again using the '+'-operator. In this example it looks like this: `CaseLinker() + TimePointLinker()`

In [42]:
combined_linker = CaseLinker() + TimePointLinker()

with session:
    PythonBinaryBatchAction(
        inputs1Selector=mr_selector,
        inputs2Selector=ct_selector,
        inputLinker=combined_linker,
        actionTag="example4",
        generateCallable=pair_two_images,
        defaultoutputextension="txt"
    ).do()

2024-11-06 12:57:15,180 [INFO] Starting action: PythonBinaryBatchAction_example4 (UID: 73c618ac-efc9-4e31-af42-19cc0a373fba) ...
2024-11-06 12:57:15,182 [INFO] Starting action: pair_two_images (UID: 56da56ca-b272-491f-8963-7a4d089b0de8) ...
2024-11-06 12:57:15,185 [INFO] Finished action: pair_two_images (UID: 56da56ca-b272-491f-8963-7a4d089b0de8) -> SUCCESS
2024-11-06 12:57:15,186 [INFO] Starting action: pair_two_images (UID: 1e63a01e-d964-42f6-bbca-f3ab99b9d3e7) ...
2024-11-06 12:57:15,188 [INFO] Finished action: pair_two_images (UID: 1e63a01e-d964-42f6-bbca-f3ab99b9d3e7) -> SUCCESS
2024-11-06 12:57:15,189 [INFO] Finished action: PythonBinaryBatchAction_example4 (UID: 73c618ac-efc9-4e31-af42-19cc0a373fba) -> SUCCESS
2024-11-06 12:57:15,195 [INFO] Successful actions: 4.
2024-11-06 12:57:15,196 [INFO] Skipped actions: 0.
2024-11-06 12:57:15,196 [INFO] Failed actions: 0.
2024-11-06 12:57:15,197 [INFO] Session finished. Feed me more...


*Result*: We can find the results in the folder ```examples/output/example_session/example4```. When looking into the text files we now find the following matches:

For pat2 we get the same results a in example 3:
- ```Input 1: pat2_TP1_MR.txt  Input 2: pat2_TP1_CT.txt.```

For pat1, there is now also only 1 match, based on case AND timepoint: 
- ```Input 1: pat1_TP1_MR.txt  Input 2: pat1_TP1_CT.txt```
 

# Example 5: 

In all previous examples, we have run one action after another in separate steps. When actions are consecutively executed like that, the output can be overwhelming at times. 

Another more elegant option is to run all the actions in an automated way by using the ```run_batches``` command. This will perform the same steps and produce the same results as above, however, this time with a more user-friendly output.

In this example we are creating a new workflow ```session2``` to run all the previously described actions in an automated way by usin the ```run_batches```command. 

Technical note: The python package rich is required for this option.

In [48]:
###############################################################################
# Initialize session with existing Artefacts
###############################################################################
session2 =  workflow.initSession(bootstrapArtefacts=os.path.join(os.getcwd(),'output', 'bootstrap.avid'),
                                sessionPath=os.path.join(os.getcwd(),'output', 'output2.avid'),
                                name = "example_session2",
                                expandPaths=True,
                                debug=True,
                                autoSave = True)



In [50]:
with session2:
    PythonUnaryBatchAction(
        inputSelector=allValid_selector,
        generateCallable=write_filename,
        actionTag="example1",
        defaultoutputextension="txt"
    )
    PythonUnaryBatchAction(
        inputSelector=pat1_selector,
        actionTag="example2",
        generateCallable=extend_content,
        defaultoutputextension="txt"
    )
    PythonBinaryBatchAction(
        inputs1Selector=mr_selector,
        inputs2Selector=ct_selector,
        inputLinker=CaseLinker(),
        actionTag="example3",
        generateCallable=pair_two_images,
        defaultoutputextension="txt"
    )
    PythonBinaryBatchAction(
        inputs1Selector=mr_selector,
        inputs2Selector=ct_selector,
        inputLinker=combined_linker,
        actionTag="example4",
        generateCallable=pair_two_images,
        defaultoutputextension="txt"
    )

    session2.run_batches()

2024-11-06 13:06:54,828 [INFO] Successful actions: 0.
2024-11-06 13:06:54,829 [INFO] Skipped actions: 0.
2024-11-06 13:06:54,830 [INFO] Failed actions: 0.
2024-11-06 13:06:54,831 [INFO] Session finished. Feed me more...


AttributeError: 'Session' object has no attribute 'run_batches'